In [1]:
import pandas as pd
from models import *
from generator import generate_balanced_dataset, draw_shapes
import numpy as np
from PIL import Image

In [2]:
data, metadata = generate_balanced_dataset(100)

In [3]:
data = pd.DataFrame(data)

In [4]:
metadata = pd.DataFrame(metadata)

In [5]:
metadata

,circle_center_x,circle_center_y,circle_radius,rect_tl_x,rect_tl_y,rect_br_x,rect_br_y
0,0.125000,-0.109375,-0.656250,0.203125,-0.593750,0.937500,0.062500
1,-0.671875,-0.343750,-0.765625,-0.218750,0.546875,0.421875,0.953125
2,0.171875,-0.578125,-0.718750,-0.203125,-0.687500,0.437500,-0.234375
3,0.187500,-0.296875,-0.687500,-0.843750,-0.687500,-0.515625,0.000000
4,-0.375000,0.734375,-0.750000,-0.750000,0.593750,-0.265625,0.984375
...,...,...,...,...,...,...,...
295,-0.625000,0.484375,-0.765625,-0.890625,0.093750,-0.218750,0.718750
296,0.406250,-0.671875,-0.828125,0.156250,-0.906250,0.609375,-0.500000
297,0.406250,0.546875,-0.812500,0.218750,0.281250,0.859375,0.781250
298,-0.312500,0.218750,-0.812500,-0.812500,-0.171875,-0.046875,0.406250


In [6]:
image_paths = [item for item in data[0]]
images = []

for path in image_paths:
    try:
        img = Image.open(path).convert('RGB')  # Convert to RGB to ensure consistency
        img = np.array(img)
        img_tensor = torch.tensor(img, dtype=torch.float32)  # Convert to PyTorch tensor
        images.append(img_tensor)
    except Exception as e:
        print(f"Error loading image {path}: {e}")

labels = torch.tensor(data[1], dtype=torch.float32)

In [7]:
draw_shapes( images[0][0], *metadata.iloc[0],)g